# Generation of new return period flood maps based on % change in peak flow

### Step 0: Import packages to work with and set up folder pathways

In [168]:
from pathlib import Path

import numpy as np
import pandas
import rasterio
import scipy

from rasterio.warp import calculate_default_transform, reproject, Resampling

In [169]:
# Define the directory containing your TIFF files - define hazards_path first to specify where the TIFF files are located.

hazards_path = Path("/Users/robynhaggis/Documents/Geospatial_analysis/Processed_data/Hazards")
output_path = Path("/Users/robynhaggis/Documents/Geospatial_analysis/Processed_data")
jamaica_metric_grid_crs = "EPSG:3448"

print(hazards_path)

/Users/robynhaggis/Documents/Geospatial_analysis/Processed_data/Hazards


### Step 1: Read in return period maps

In [170]:
# List all files in the directory to check that it is pointing to the right place
all_files = list(hazards_path.iterdir())
display("All files in directory:", [f.name for f in all_files])

'All files in directory:'

['JM_FLRF_UD_Q50_RD_02.tfw',
 'JM_FLRF_UD_Q10_RD_02_aligned.tif',
 'JM_FLRF_UD_Q50_RD_02.tif',
 'JM_FLRF_UD_Q1500_RD_02.tif.ovr',
 'JM_FLRF_UD_Q50_RD_02.tif.xml',
 '.DS_Store',
 'JM_FLRF_UD_Q50_RD_02_flow0.8.tif',
 'JM_FLRF_UD_Q50_RD_02_flow0.9.tif',
 'JM_FLRF_UD_Q100_RD_02_flow0.9.tif',
 'JM_FLRF_UD_Q20_RD_02_aligned.tif',
 'JM_FLRF_UD_Q100_RD_02.tif.xml',
 'JM_FLRF_UD_Q20_RD_02.tif.xml',
 'JM_FLRF_UD_Q5_RD_02-aligned.tif',
 'JM_FLRF_UD_Q100_RD_02_flow0.8.tif',
 'JM_FLRF_UD_Q20_RD_02.tif.aux.xml',
 'JM_FLRF_UD_Q500_RD_02.tfw',
 'JM_FLRF_UD_Q20_RD_02_flow0.6.tif',
 'JM_FLRF_UD_Q500_RD_02.tif',
 'JM_FLRF_UD_Q1500_RD_02.tif.xml',
 'JM_FLRF_UD_Q50_RD_02.tif.ovr',
 'JM_FLRF_UD_Q2_RD_02-aligned.tif',
 'JM_FLRF_UD_Q10_RD_02_flow0.6.tif',
 'JM_FLRF_UD_Q20_RD_02.tif',
 'JM_FLRF_UD_Q20_RD_02.tif.ovr',
 'JM_FLRF_UD_Q100_RD_02.tif.ovr',
 'JM_FLRF_UD_Q20_RD_02.tfw',
 'JM_FLRF_UD_Q10_RD_02_flow0.8.tif',
 'JM_FLRF_UD_Q2_RD_02_aligned.tif',
 'JM_FLRF_UD_Q50_RD_02_flow0.95.tif',
 'JM_FLRF_UD_Q10_RD_02

In [189]:
# Read in river flood maps: found via processed data -> hazards -> Global Flood Map -> Jamaica -> Fluvial -> Raw Depths
# Tifs as follows: [Q20_RD_02.tif; Q50_RD_02.tif; Q100_RD_02.tif; Q200_RD_02.tif; Q500_RD_02.tif]  


# List all TIFF files in the directory of tiff hazards - list TIFF files in river_flood_tiff_files to make sure the path is correct.
river_flood_tiff_files = list(hazards_path.glob('*.tif'))

# Define the function to read a TIFF file
def read_rp_map(fname: Path) -> np.ndarray:
    """Read flood map TIFF file into a NumPy array."""
    with rasterio.open(fname) as dataset:
        data = dataset.read(1)
        data[data == dataset.nodata] = 0  # Replace no-data values with 0
    return data
    
# Read each TIFF file and store the data in a dictionary
river_flood_maps = {file.stem: read_rp_map(file) for file in river_flood_tiff_files}

# Print the names of the loaded files to verify - it prints the names of all the loaded tif files
display("Loaded flood maps:", river_flood_maps.keys())

'Loaded flood maps:'

dict_keys(['JM_FLRF_UD_Q10_RD_02_aligned', 'JM_FLRF_UD_Q50_RD_02', 'JM_FLRF_UD_Q50_RD_02_flow0.8', 'JM_FLRF_UD_Q50_RD_02_flow0.9', 'JM_FLRF_UD_Q100_RD_02_flow0.9', 'JM_FLRF_UD_Q20_RD_02_aligned', 'JM_FLRF_UD_Q5_RD_02-aligned', 'JM_FLRF_UD_Q100_RD_02_flow0.8', 'JM_FLRF_UD_Q20_RD_02_flow0.6', 'JM_FLRF_UD_Q500_RD_02', 'JM_FLRF_UD_Q2_RD_02-aligned', 'JM_FLRF_UD_Q10_RD_02_flow0.6', 'JM_FLRF_UD_Q20_RD_02', 'JM_FLRF_UD_Q10_RD_02_flow0.8', 'JM_FLRF_UD_Q2_RD_02_aligned', 'JM_FLRF_UD_Q50_RD_02_flow0.95', 'JM_FLRF_UD_Q10_RD_02_flow0.9', 'JM_FLRF_UD_Q500_RD_02_aligned', 'JM_FLRF_UD_Q10_RD_02', 'JM_FLRF_UD_Q10_RD_02_flow0.95', 'JM_FLRF_UD_Q200_RD_02', 'JM_FLRF_UD_Q200_RD_02_aligned', 'JM_FLRF_UD_Q50_RD_02_aligned', 'JM_FLRF_UD_Q5_RD_02', 'JM_FLRF_UD_Q100_RD_02_aligned', 'JM_FLRF_UD_Q20_RD_02_flow0.9', 'JM_FLRF_UD_Q20_RD_02_flow0.8', 'JM_FLRF_UD_Q20_RD_02_flow0.95', 'JM_FLRF_UD_Q1500_RD_02', 'JM_FLRF_UD_Q50_RD_02_flow0.6', 'JM_FLRF_UD_Q100_RD_02_flow0.6', 'JM_FLRF_UD_Q5_RD_02_aligned', 'JM_FLRF_UD_Q

### Step 2: Read in the file metadata

In [172]:
# Read metadata from an existing GeoTIFF file specified by fname. Extract the following:
# -- CRS: The coordinate reference system.
# -- Width and Height: The raster’s dimensions (in pixels).
# -- Transform: The affine transformation matrix for georeferencing (e.g., linking pixel coordinates to geographic space).
# -- Purpose: This function does not modify the TIFF file; it only reads and returns the metadata.

def read_metadata(fname):
    with rasterio.open(fname) as dataset:
        crs = dataset.crs
        width = dataset.width
        height = dataset.height
        transform = dataset.transform
    return crs, width, height, transform


def reproject_raster(input_path, output_path, target_crs, target_shape=None, target_transform=None):
    """Reproject and resample a raster to a common CRS, resolution, and extent."""
    with rasterio.open(input_path) as src:
        if target_transform and target_shape:
            transform = target_transform
            width, height = target_shape
        else:
            transform, width, height = calculate_default_transform(
                src.crs, target_crs, src.width, src.height, *src.bounds
            )
        
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': target_crs,
            'transform': transform,
            'width': width,
            'height': height,
        })
        
        with rasterio.open(output_path, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=target_crs,
                    resampling=Resampling.nearest
                )

In [173]:
# Save the data (a NumPy array) to a new GeoTIFF file at the specified fname. It includes the provided:
# -- CRS: Specifies the coordinate reference system for georeferencing.
# -- Transform: Links the raster data to geographic coordinates.
# -- Uses "w" mode to write a new file. If fname already exists, it will be overwritten.
# -- Compresses the data using LZW compression to save disk space.


### Check to see if I need to specify new file names

def save_to_tif(data, fname, crs, transform):
    with rasterio.open(
        fname,
        "w",
        driver="GTiff",
        height=data.shape[0],
        width=data.shape[1],
        count=1,
        dtype=data.dtype,
        crs=crs,
        transform=transform,
        compress="lzw",
    ) as dataset:
        dataset.write(data, 1)

### Step 3: Interpolate between flood maps in order to develop tifs of missing return periods (RP2,5,10)

In [174]:
# Read all the different return period maps as TIFF files and return their data as a dictionary of NumPy arrays.

# Function Signature:
  # Input:
	# - rp_maps: A dictionary where:
	# - - Keys are float values representing return periods (e.g., 2, 5, 10, etc.).
	# - - Values are Path objects representing the file paths to the corresponding flood map TIFF files.
  # Output:
	# - A dictionary (rp_data) where:
	# - - Keys are the same return periods (float values).
	# - - Values are NumPy arrays (np.ndarray) containing the raster data from the corresponding TIFF files.


# def read_rp_maps(rp_maps: dict[float, Path]) -> dict[float, np.ndarray]:
  #   """Read flood map tiffs to dict of ndarrays
   #  """
    # rp_data: dict[float, np.ndarray] = {}
    # for rp, fname in rp_maps.items():
      #   rp_data[rp] = read_rp_map(fname)
    # return rp_data

def read_rp_maps(rp_maps: dict[float, Path]) -> dict[float, np.ndarray]:
    """Read flood map TIFFs to dict of ndarrays, validating file existence"""
    rp_data: dict[float, np.ndarray] = {}
    for rp, fname in rp_maps.items():
        if not Path(fname).exists():
            raise FileNotFoundError(f"Input file not found: {fname}")
        rp_data[rp] = read_rp_map(fname)
    return rp_data

In [175]:
def interpolate_depth(rp: float, rp_l: float, depth_l: np.ndarray, rp_u: float, depth_u: np.ndarray) -> np.ndarray:
    """Interpolate between two flood map ndarrays
    """
    rp_factor = (np.log(rp) - np.log(rp_l)) / (np.log(rp_u) - np.log(rp_l))
    depth = depth_l + ((depth_u - depth_l) * rp_factor)

    return depth

In [176]:
def pick_upper_lower_rps(rp: float, rps: list[float]) -> tuple[float]:
    bin_index = np.searchsorted(rps, rp, side="left")
    rp_l = rps[bin_index - 1]
    rp_u = rps[bin_index]
    return rp_l, rp_u

In [177]:
def calculate_rp_maps(rps_to_calculate, rps_input):
    # Read all data
    depths = read_rp_maps(rps_input)
    baseline_rps = list(depths.keys())
    # Read metadata
    ## The 1e-3 I think means that it is not quite 0, just above it. Hence possibly why we get the 0.001 as an output of the next box.
    crs, width, height, transform = read_metadata(rps_input[baseline_rps[0]])
    depths[1e-3] = np.zeros((height, width))
    depths[2.0] = np.zeros((height, width))
    depths[1e6] = depths[max(baseline_rps)]
    baseline_rps = [1e-3, 2] + sorted(baseline_rps) + [1e6]

# Calculate and save interpolated depths for new return periods

    for rp, output_fname in rps_to_calculate.items():
        rp_l, rp_u = pick_upper_lower_rps(rp, baseline_rps)
        print(rp_l, rp_u)
        depth = interpolate_depth(rp, rp_l, depths[rp_l], rp_u, depths[rp_u])

# Save the output depth map to the specified file

        save_to_tif(depth, output_fname, crs, transform)

In [178]:
# Develop tifs for the return periods that are missing: RP2, RP5, RP10. 

# Input return periods and file paths

rps_input = {
    20: hazards_path / "JM_FLRF_UD_Q20_RD_02.tif",
    50: hazards_path / "JM_FLRF_UD_Q50_RD_02.tif",
    100: hazards_path / "JM_FLRF_UD_Q100_RD_02.tif",
    200: hazards_path / "JM_FLRF_UD_Q200_RD_02.tif",
    500: hazards_path / "JM_FLRF_UD_Q500_RD_02.tif"
}

# Define output return periods

rps_to_calculate = {
    2: hazards_path / "JM_FLRF_UD_Q2_RD_02_aligned.tif",
    5: hazards_path / "JM_FLRF_UD_Q5_RD_02_aligned.tif",
    10: hazards_path / "JM_FLRF_UD_Q10_RD_02_aligned.tif"
}

# Define the reference raster
reference_raster = hazards_path / "JM_FLRF_UD_Q20_RD_02.tif"

# Reproject and align all input rasters
aligned_rasters = {}
with rasterio.open(reference_raster) as ref:
    ref_crs = ref.crs
    ref_transform = ref.transform
    ref_shape = (ref.height, ref.width)
    
    for rp, raster_path in rps_input.items():
        output_path = hazards_path / f"{raster_path.stem}_aligned.tif"
        reproject_raster(raster_path, output_path, ref_crs, ref_shape, ref_transform)
        aligned_rasters[rp] = output_path

# Update rps_input to use aligned rasters
rps_input = aligned_rasters

# Call the calculate_rp_maps function
calculate_rp_maps(rps_to_calculate, rps_input)

rp_data = read_rp_maps(rps_input)

0.001 2
2 20
2 20


In [179]:
print("Loaded return periods:", rp_data.keys())
print("Data for RP 20:", rp_data[20])  # Print the NumPy array for RP 20

Loaded return periods: dict_keys([20, 50, 100, 200, 500])
Data for RP 20: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [180]:
print("Checking input files...")
for rp, path in rps_input.items():
    if not Path(path).exists():
        print(f"Input file not found: {path}")

Checking input files...


In [181]:
output_file = hazards_path / "JM_FLRF_UD_Q2_RD_02.tif"
with rasterio.open(output_file) as dataset:
    data = dataset.read(1)
    print("Data for RP 2:", data)

Data for RP 2: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [182]:
print("Expected output files:")
for rp, path in rps_to_calculate.items():
    print(f"{rp}: {path}")

Expected output files:
2: /Users/robynhaggis/Documents/Geospatial_analysis/Processed_data/Hazards/JM_FLRF_UD_Q2_RD_02_aligned.tif
5: /Users/robynhaggis/Documents/Geospatial_analysis/Processed_data/Hazards/JM_FLRF_UD_Q5_RD_02_aligned.tif
10: /Users/robynhaggis/Documents/Geospatial_analysis/Processed_data/Hazards/JM_FLRF_UD_Q10_RD_02_aligned.tif


### Step 4: Define how return period changes according to percentage change in peak flow

In [183]:
# Top row outlines change in peak flow. 
## Subsequent rows define the return period and what it becomes.
### Assume for return period of 2 or below, flood depth is 0, i.e. the infrastructure asset experiences no damage. 


#Tom_flow_reductions = pandas.DataFrame({
#    'reduction_percent': [5, 10, 20, 30],
#    'rp2': [2.4, 2.9, 4.8, 7.6],
#    'rp10': [13,18,33,55.5],
#    'rp50': [67,90,173,297],
#    'rp100': [133,180,346,594],
#    'rp200': [266,359,690,1188],
#    'rp1000': [1327,1788,3419,5941],
#})

# Need to add in 30 and check them as 2.3 and 5 are the same
CCRA_flow_reductions = pandas.DataFrame({
    'reduction_percent': [5, 10, 20, 40],
    'rp2': [2.4, 3.2, 6.8, 196],
    'rp2.3': [2.8, 3.7, 7.9, 169],
    'rp5': [6.5, 8.9, 19, 235],
    'rp10': [13,19,43,473],
    'rp25': [35, 51, 123, 1330],
    'rp50': [72,107,268,2967],
    'rp100': [147,224,582, 6648],
    #'rp200': [266,359,690,1188],
    'rp500': [772,1229,3441,43094],
    'rp1000': [1571,2544,7345,95943],
})

# proportion of baseline flow
CCRA_flow_reductions['flow'] = (1 - CCRA_flow_reductions.reduction_percent / 100)



# The below code is used to interpolate based on the current return periods (2,12,50,100,500,1000) to define a new return period (20)
## interpolate RP20 values
known_rps = [2, 10, 50, 100, 200, 1000]
flows = CCRA_flow_reductions['flow'].values

interpolator = scipy.interpolate.RegularGridInterpolator(
    (known_rps, flows),
    CCRA_flow_reductions[['rp2', 'rp10', 'rp50', 'rp100', 'rp500', 'rp1000']].values.T,
    method='cubic'
)

rps_new = [[20]]
CCRA_flow_reductions['rp20'] = interpolator((rps_new, [flows]))[0]

CCRA_flow_reductions[['flow', 'rp2', 'rp10', 'rp20', 'rp50', 'rp100', 'rp500', 'rp1000']]

,flow,rp2,rp10,rp20,rp50,rp100,rp500,rp1000
0,0.95,2.4,13,27.797332,72,147,772,1571
1,0.90,3.2,19,40.894524,107,224,1229,2544
2,0.80,6.8,43,97.099839,268,582,3441,7345
3,0.60,196.0,473,1003.870214,2967,6648,43094,95943


In [188]:
display(CCRA_flow_reductions.head())

,reduction_percent,rp2,rp2.3,rp5,rp10,rp25,rp50,rp100,rp500,rp1000,flow,rp20
0,5,2.4,2.8,6.5,13,35,72,147,772,1571,0.95,27.797332
1,10,3.2,3.7,8.9,19,51,107,224,1229,2544,0.90,40.894524
2,20,6.8,7.9,19.0,43,123,268,582,3441,7345,0.80,97.099839
3,40,196.0,169.0,235.0,473,1330,2967,6648,43094,95943,0.60,1003.870214


In [184]:

for _, row in CCRA_flow_reductions.iterrows():
    flow = row.flow
    
    # declare desired output RP and filename
    rps_to_calculate = {
        10: hazards_path / f"JM_FLRF_UD_Q10_RD_02_flow{flow}.tif",
        20: hazards_path / f"JM_FLRF_UD_Q20_RD_02_flow{flow}.tif",
        50: hazards_path / f"JM_FLRF_UD_Q50_RD_02_flow{flow}.tif",
        100: hazards_path / f"JM_FLRF_UD_Q100_RD_02_flow{flow}.tif",
    }
    # use baseline RP maps "mislabelled" as after change in flow
    rps_input = {
        row.rp2: hazards_path / "JM_FLRF_UD_Q2_RD_02_aligned.tif",
        row.rp10: hazards_path / "JM_FLRF_UD_Q10_RD_02_aligned.tif",
        row.rp20: hazards_path / "JM_FLRF_UD_Q20_RD_02_aligned.tif",
        row.rp50: hazards_path / "JM_FLRF_UD_Q50_RD_02_aligned.tif",
        row.rp100: hazards_path / "JM_FLRF_UD_Q100_RD_02_aligned.tif",
    }
    calculate_rp_maps(rps_to_calculate, rps_input)


2.4 13.0
13.0 27.797332285797815
27.797332285797815 72.0
72.0 147.0
3.2 19.0
19.0 40.894523984030975
40.894523984030975 107.0
40.894523984030975 107.0
6.8 43.0
6.8 43.0
43.0 97.09983855594614
97.09983855594614 268.0
2 196.0
2 196.0
2 196.0
2 196.0


In [185]:
for rp, path in {**rps_to_calculate, **rps_input}.items():
    if not Path(path).exists():
        print(f"File not found: {path}")

In [186]:
print("All files in hazards_path:")
for file in hazards_path.iterdir():
    print(file.name)

All files in hazards_path:
JM_FLRF_UD_Q50_RD_02.tfw
JM_FLRF_UD_Q10_RD_02_aligned.tif
JM_FLRF_UD_Q50_RD_02.tif
JM_FLRF_UD_Q1500_RD_02.tif.ovr
JM_FLRF_UD_Q50_RD_02.tif.xml
.DS_Store
JM_FLRF_UD_Q50_RD_02_flow0.8.tif
JM_FLRF_UD_Q50_RD_02_flow0.9.tif
JM_FLRF_UD_Q100_RD_02_flow0.9.tif
JM_FLRF_UD_Q20_RD_02_aligned.tif
JM_FLRF_UD_Q100_RD_02.tif.xml
JM_FLRF_UD_Q20_RD_02.tif.xml
JM_FLRF_UD_Q5_RD_02-aligned.tif
JM_FLRF_UD_Q100_RD_02_flow0.8.tif
JM_FLRF_UD_Q20_RD_02.tif.aux.xml
JM_FLRF_UD_Q500_RD_02.tfw
JM_FLRF_UD_Q20_RD_02_flow0.6.tif
JM_FLRF_UD_Q500_RD_02.tif
JM_FLRF_UD_Q1500_RD_02.tif.xml
JM_FLRF_UD_Q50_RD_02.tif.ovr
JM_FLRF_UD_Q2_RD_02-aligned.tif
JM_FLRF_UD_Q10_RD_02_flow0.6.tif
JM_FLRF_UD_Q20_RD_02.tif
JM_FLRF_UD_Q20_RD_02.tif.ovr
JM_FLRF_UD_Q100_RD_02.tif.ovr
JM_FLRF_UD_Q20_RD_02.tfw
JM_FLRF_UD_Q10_RD_02_flow0.8.tif
JM_FLRF_UD_Q2_RD_02_aligned.tif
JM_FLRF_UD_Q50_RD_02_flow0.95.tif
JM_FLRF_UD_Q10_RD_02_flow0.9.tif
JM_FLRF_UD_Q200_RD_02.tif.xml
JM_FLRF_UD_Q500_RD_02_aligned.tif
JM_FLRF_UD_Q10